# TP7 QuickSort en OpenMP
## M1 informatique, Université d'Orléans 2021/2022

L’objectif de ce TP est de paralléliser l’algorithme de tri appelé QuickSort en suivant les algorithmes vus
en cours et décrits dans le chapitre 9 du livre ”Introduction to Parallel Computing” de A. Grama, A. Gupta, G.
Karypis et V.Kumar (livre disponible [ici](http://srmcse.weebly.com/uploads/8/9/0/9/8909020/introduction_to_parallel_computing_second_edition-ananth_grama..pdf))

Vous disposez de la version séquentielle sur Celene.

### 1. La décomposition d'une étape

Dans cette première partie vous allez écrire les fonctions réalisant les opérations élémentaires d’une étape de l’algorithme QuickSort. 

#### 1.1 La fonction Partitionnement 

Cette fonction permet de répartir les éléments à droite ou à gauche du pivot. Chaque thread devra effectuer le partitionnement d’un morceau du tableau global. La signature à respecter est la suivante

In [1]:
void Partitionnement(int* tab, int n, int pivot, int nb_threads, int* s, int* r);

* tab est le tableau de données
* n sa longueur
* pivot est le pivot que vous choisirez avec la méthode de votre choix
* nb_threads est le nombre de threads souhaité
* s est un tableau tel que s[i] est le nombre d’éléments plus petits ou égaux au pivot pour 
le ième morceau du tableau 0 ≤ i < nb threads
* r est un tableau tel que r[i] est le nombre d’éléments plus grand que le pivot pour 
le ième morceau du tableau 0 ≤ i < nb threads

SyntaxError: invalid syntax (2271471314.py, line 1)

```c
Void Partitionnement(int* tab, int n, int pivot, int nb_threads, int* s, int* r){
    //srand (time(NULL));
    //rand()%100;
    int chunk = n/nb_threads;
    int reste = n%nb_threads;
#pragma omp parallel num_threads(nb_threads) shared(s,r,tab,pivot,n)
    {
        int pid = omp_get_thread_num();
        int debut = chunk*pid + min(pid,reste);
        int fin = debut+chunk;
        if(pid<reste){
            fin++;
        }
        for(int i = debut; i<fin;i++){
            if(tab[i]<=pivot){
                swap(tab[i],tab[s[pid]]);
                s[pid]++;
            }

        }
        r[pid]=chunk-s[pid];
    }

}
```

#### 1.2 La fonction SommePrefixe 

Cette fonction permet de déterminer les tableaux *somme_left* et *somme_right* de taille nb_threads+1 tels que **somme_left[i]** (respectivement **somme_right[i]**) est le nombre d’éléments plus petits
ou égaux (respectivement plus grands) que le pivot et qui ont été trouvés par les threads qui précédent le
**thread i** (somme_left[0] = somme_right[0]=0). 
La signature à respecter est la suivante


```c
void SommePrefixe(int* s, int* r, int* somme_left, int* somme_right, int nb_threads);

* s est le tableau calculé précedemment avec le nombre des éléments plus petits ou égaux au pivot
* r idem pour le nombre des éléments pls grands que le pivot
```

```c
void SommePrefixe(int* s, int* r, int* somme_left, int* somme_right, int nb_threads){
#pragma omp parallel num_threads(nb_threads)
    {
        int pid = omp_get_thread_num();
//init 0
        if(pid==0){
            somme_left[0]=0;
            somme_right[0]=0;
        }else{
            somme_right[pid]=r[pid-1];
            somme_left[pid]=s[pid-1];
        }
#pragma omp barrier
        for (double i = 1; i < log2(nb_threads);i*=2) {
            if(pid>=i){
                somme_left[pid]+=somme_left[pid-i];
                somme_right[pid]+=somme_right[pid-i];
            }
#pragma omp barrier
        }
    }
}
```

#### 1.3 La fonction Rearrangement 

Cette fonction permet de reconstruire correctement un nouveau tableau tel que tous les éléments plus petits ou égaux au pivot soient à gauche dans le tableau et les autres à droite. La signature à respecter est la suivante

```c
void Rearrangement(int* somme_left, int* somme_right, int* tab, int* res, int n, int nb_threads);

* res est le tableau résultat du réarrangement.
* les autres paramètres sont similaires aux fonctions précédentes.
```

```c
void Rearrangement(int* somme_left, int* somme_right, int* tab, int* res, int n, int nb_threads){
    int chunk = n/nb_threads;
    int reste = n%nb_threads;
#pragma omp parallel num_threads(nb_threads)
    {
        int pid = omp_get_thread_num();
        int debut = chunk*pid + min(pid,reste);
        int fin = debut+chunk;
        int pivot = somme_left[pid+1];
        int fin_left = somme_left[nb_threads];
        if(pid<reste){
            fin++;
        }
        for(int i = debut; i<fin;i++){
            if(i<=pivot){
                res[somme_left[pid]++]=tab[i];
            }else{
                res[fin_left+somme_right[pid]++]=tab[i]
            }
        }
}
}
```

### 2. Mise en place de l’algorithme récursif

Cette partie consiste à finaliser votre développement et à mettre en place l’algorithme récursif qui permet de
réaliser le tri. Vous êtes libres de réorganiser votre code comme vous le souhaitez.

Vous devrez utiliser les tâches OpenMP afin de mettre en place la récursivité. Faîtes attention à lancer qu'une seule fois les tâches en indiquant le nombre de threads qu'elles utiliseront. Lors d'une étape sur un tableau de taille **n** sur **A threads** si **m** est le nombre d'éléments plus petits ou égaux au pivot alors on exécutera le QuickSort des éléments à gauche du pivot sur **$\bf \lceil\frac{m\times A}{n}+0.5\rceil$ threads**. 

In [ ]:
#pragma omp single 
{
#pragma omp task
    // task1....
#pragma omp single
    // task2...
}